#install

In [2]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express
!pip install Shapely
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-mb8i5n8t
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-mb8i5n8t


In [57]:
from shapely.geometry import Point,Polygon
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt 
import folium
from folium import plugins
from folium.plugins import HeatMap
import plotly_express as px
from google.colab import drive
from IPython.display import display
import glob
import re
import plotly_express as px
import seaborn as sns
import os
from os.path import exists
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Read

In [4]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019'

trips_2019_1_10.csv  trips_2019_2_19.csv  trips_2019_3_2.csv
trips_2019_1_11.csv  trips_2019_2_1.csv   trips_2019_3_30.csv
trips_2019_1_12.csv  trips_2019_2_20.csv  trips_2019_3_31.csv
trips_2019_1_13.csv  trips_2019_2_21.csv  trips_2019_3_3.csv
trips_2019_1_14.csv  trips_2019_2_22.csv  trips_2019_3_4.csv
trips_2019_1_15.csv  trips_2019_2_23.csv  trips_2019_3_5.csv
trips_2019_1_16.csv  trips_2019_2_24.csv  trips_2019_3_6.csv
trips_2019_1_17.csv  trips_2019_2_25.csv  trips_2019_3_7.csv
trips_2019_1_18.csv  trips_2019_2_26.csv  trips_2019_3_8.csv
trips_2019_1_19.csv  trips_2019_2_27.csv  trips_2019_3_9.csv
trips_2019_1_1.csv   trips_2019_2_28.csv  trips_2019_4_01.csv
trips_2019_1_20.csv  trips_2019_2_2.csv   trips_2019_4_02.csv
trips_2019_1_21.csv  trips_2019_2_3.csv   trips_2019_4_03.csv
trips_2019_1_22.csv  trips_2019_2_4.csv   trips_2019_4_04.csv
trips_2019_1_23.csv  trips_2019_2_5.csv   trips_2019_4_05.csv
trips_2019_1_24.csv  trips_2019_2_6.csv   trips_2019_4_06.csv
trips_2019_1_25.

In [5]:
tripPath = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019'
districtPath = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/District_NE.geojson'

In [6]:
district_NE = gpd.read_file(districtPath)
district_NE

,AMP_CODE,PRV_CODE,AMP_NAME_T,AMP_NAME_E,Shape_Leng,Shape_Area,geometry
0,3001,30,เมืองนครราชสีมา,Mueang Nakhon Ratchasima,160600.306403,8.856562e+08,"POLYGON ((102.18341 15.13798, 102.18298 15.137..."
1,3002,30,ครบุรี,Khon Buri,261136.836300,2.168627e+09,"POLYGON ((102.15081 14.61461, 102.15115 14.614..."
2,3003,30,เสิงสาง,Soeng Sang,181829.710406,1.097496e+09,"POLYGON ((102.49675 14.50541, 102.49686 14.505..."
3,3004,30,คง,Khong,208115.123225,7.534105e+08,"POLYGON ((102.27233 15.53663, 102.27538 15.536..."
4,3005,30,บ้านเหลื่อม,Ban Lueam,88918.433064,2.569881e+08,"POLYGON ((102.12032 15.64497, 102.12072 15.644..."
...,...,...,...,...,...,...,...
316,3902,39,นากลาง,Na Klang,190622.272867,7.934911e+08,"POLYGON ((102.13153 17.49852, 102.13277 17.498..."
317,3903,39,โนนสัง,Non Sang,127457.559383,7.620915e+08,"POLYGON ((102.62819 17.01421, 102.62874 17.013..."
318,3904,39,ศรีบุญเรือง,Si Bun Rueang,253122.478708,1.053086e+09,"POLYGON ((102.06074 17.20408, 102.06092 17.204..."
319,3905,39,สุวรรณคูหา,Suwannakhuha,142751.437069,6.590840e+08,"POLYGON ((102.31839 17.67681, 102.31874 17.667..."


#Algorithm

In [8]:
def check_district(lon,lat):
  data = zip(district_NE.geometry,district_NE.AMP_NAME_T)
  point = Point(lon,lat)
  for index in data:
    if point.within(index[0]):
      return index[1]
  return False

In [9]:
check_district(102.151459,14.915597)

'เมืองนครราชสีมา'

#Feature

In [ ]:
Path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/Feature_trips'
days = range(1,32)
month = 1
for day in days:
    export_filename = Path + f'/trips_2019_{month}_{day}.csv'

      #### check file exists
    if os.path.exists(export_filename):
      print(f'trips_2019_{month}_{day}.csv already taken!!')
      continue
    else:
      with open(export_filename, 'w') as fp:
        pass
      if os.path.exists(export_filename):
        print(f'trips_2019_{month}_{day}.csv creating...')
        pass

    filenames = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019/' + f'trips_2019_{month}_{day}.csv'
    trip = pd.read_csv(filenames)

    #district
    cood = zip(trip.lon,trip.lat)
    i = 0
    for data in cood:
      trip.loc[i,'district'] = check_district(data[0],data[1])
      i = i + 1
    
    #days
    trip.time_stamp = trip.time_stamp.astype("datetime64")
    if trip.loc[0].time_stamp.day_name() == 'Monday':
      trip['day'] = 0
    if (trip.loc[0].time_stamp.day_name() == 'Tuesday') or (trip.loc[0].time_stamp.day_name() == 'Wednesday') or (trip.loc[0].time_stamp.day_name() == 'Thursday'):
      trip['day'] = 1
    if trip.loc[0].time_stamp.day_name() == 'Friday':
      trip['day'] = 2
    if trip.loc[0].time_stamp.day_name() == 'Saturday' or (trip.loc[0].time_stamp.day_name() == 'Sunday'):
      trip['day'] = 3

    trip.to_csv(export_filename, encoding='utf-8', index = False)
    print(f'trips {month} {day} done')


trips_2019_1_1.csv creating...
